In [72]:
from chainer import Chain, Variable, optimizers, iterators, serializers, training, cuda
from chainer.training import extensions
from chainer.datasets import mnist
import chainer.functions as F
import chainer.links as L
import chainer

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

In [86]:
DIRNAME = 'resized_32_images/'
SIZE = 32
CHANNEL = 3
files = os.listdir(DIRNAME)
N_data = len(files)
train_data = np.zeros((N_data, CHANNEL, SIZE, SIZE))
for i,file in enumerate(files):
    im = Image.open(DIRNAME + file)
    train_data[i] = np.array(im).reshape(CHANNEL,SIZE,SIZE)
train_data = (train_data - 127.5) / 127.5
train_data = train_data.astype(np.float32)

In [87]:
batchsize = 64
dis_itr = iterators.SerialIterator(train_data, batchsize)
initializer = chainer.initializers.HeNormal()

In [88]:
class Generator(Chain):
    
    def __init__(self):
        super(Generator, self).__init__()
        with self.init_scope():
            self.fc1 = L.Linear(100, 1024, initialW = initializer)
            self.fc2 = L.Linear(1024, 128 * 8 * 8, initialW = initializer)
            self.dc3 = L.Deconvolution2D(128, 64, 4, 2, 1, initialW = initializer)
            self.dc4 = L.Deconvolution2D(64, 3, 4, 2, 1, initialW = initializer)
            
            self.bn1 = L.BatchNormalization(1024)
            self.bn2 = L.BatchNormalization(128 * 8 * 8)
            self.bn3 = L.BatchNormalization((64, 16, 16))
    def __call__(self, x):
        h = F.relu(self.bn1(self.fc1(x)))
        h = F.relu(self.bn2(self.fc2(h)))
        h = F.reshape(h, (h.shape[0], 128, 8, 8))
        h = F.relu(self.bn3(self.dc3(h)))
        h = F.tanh(self.dc4(h))
        return h

In [89]:
class Discriminator(Chain):
    
    def __init__(self):
        super(Discriminator, self).__init__()
        with self.init_scope():
            self.cv1 = L.Convolution2D(3, 64, 4, 2, initialW = initializer)
            self.cv2 = L.Convolution2D(64, 128, 4, 2, initialW = initializer)
            self.fc3 = L.Linear(None, 256, initialW = initializer)
            self.fc4 = L.Linear(256, 2, initialW = initializer)

    def __call__(self, x):
        h = F.leaky_relu(self.cv1(x), 0.2)
        h = F.leaky_relu(self.cv2(h), 0.2)
        h = F.leaky_relu(self.fc3(h), 0.2)
        h = F.dropout(h, 0.5)
        h = self.fc4(h)
        return h

In [92]:
max_epoch = 20000
GENERATED_IMAGE_PATH = 'generated_images/'
if not os.path.exists(GENERATED_IMAGE_PATH):
    os.mkdir(GENERATED_IMAGE_PATH)

In [94]:
def train():
    chainer.cuda.memory_pool.free_all_blocks()
    
    model_gen = Generator()
    model_dis = Discriminator()
    
    opt_gen = optimizers.Adam(alpha = 2e-4, beta1 = 0.5)
    opt_dis = optimizers.Adam(alpha = 2e-4, beta1 = 0.5)
    opt_gen.setup(model_gen)
    opt_dis.setup(model_dis)
    opt_gen.add_hook(chainer.optimizer.WeightDecay(0.00001))
    opt_dis.add_hook(chainer.optimizer.WeightDecay(0.00001))
    
    gpu_device = 0
    cuda.get_device(gpu_device).use()
    model_gen.to_gpu(gpu_device)
    model_dis.to_gpu(gpu_device)
    xp = cuda.cupy
    
    dis_itr.epoch = 0
    while dis_itr.epoch < max_epoch:
        real_images = xp.array(dis_itr.next())
        y = model_dis(real_images)
        loss_dis = F.softmax_cross_entropy(y, Variable(xp.zeros(batchsize).astype(np.int32)))
        
        noise = xp.array(np.random.uniform(-1, 1, (batchsize, 100)).astype(np.float32))
        generated_images = model_gen(noise)
        y = model_dis(generated_images)
        
        loss_dis += F.softmax_cross_entropy(y, Variable(xp.ones(batchsize).astype(np.int32)))
        model_dis.cleargrads()
        loss_dis.backward()
        opt_dis.update()
        
        noise = xp.array(np.random.uniform(-1, 1, (batchsize, 100)).astype(np.float32))
        y = model_dis(model_gen(noise))
        loss_gen = F.softmax_cross_entropy(y, Variable(xp.zeros(batchsize).astype(np.int32)))
        model_gen.cleargrads()
        loss_gen.backward()
        opt_gen.update()
        
        if dis_itr.is_new_epoch and dis_itr.epoch % 100 == 0:
            print("epoch:", dis_itr.epoch)
            print("loss_dis:", loss_dis, "  loss_gen:", loss_gen)
            for n,image in enumerate(generated_images[:30]):
                image = cuda.to_cpu(image.data)*127.5 + 127.5
                Image.fromarray(image.astype(np.uint8).reshape(SIZE, SIZE, CHANNEL)) \
                                                      .save(GENERATED_IMAGE_PATH+"%03d_%03d.png" % (dis_itr.epoch, n))
                    
    MODEL_SAVE_DIR = 'DCGAN1'
    if not os.path.exists(MODEL_SAVE_DIR):
        os.mkdir(MODEL_SAVE_DIR)
    serializers.save_hdf5("%s/dcgan_model_dis_%d.h5"%(MODEL_SAVE_DIR, dis_itr.epoch), model_dis)
    serializers.save_hdf5("%s/dcgan_model_gen_%d.h5"%(MODEL_SAVE_DIR, dis_itr.epoch), model_gen)
    serializers.save_hdf5("%s/dcgan_state_dis_%d.h5"%(MODEL_SAVE_DIR, dis_itr.epoch), opt_dis)
    serializers.save_hdf5("%s/dcgan_state_gen_%d.h5"%(MODEL_SAVE_DIR, dis_itr.epoch), opt_gen)

In [ ]:
train()

epoch: 100
loss_dis: variable(0.50066686)   loss_gen: variable(2.7657716)
epoch: 200
loss_dis: variable(0.38380182)   loss_gen: variable(1.9777774)
